<a href="https://colab.research.google.com/github/mylethidiem/zero-to-hero/blob/main/preAIO25/external/VisionTransformer_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Vision Transformer**